In [4]:
import sys
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pandas

sys.path.append('/kaggle/code/ConeDetectionPointpillars')
import fire
import pickle
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix, quaternion_yaw
from second.data.dataset import Dataset, register_dataset, get_dataset_class
from nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud, Box, Quaternion
from nuscenes.utils.geometry_utils import *
from second.core import box_np_ops

TRAINVAL_SPLIT_PERCENTAGE = 0.95
NameMapping = {
    'movable_object.barrier': 'barrier',
    'vehicle.bicycle': 'bicycle',
    'vehicle.bus.bendy': 'bus',
    'vehicle.bus.rigid': 'bus',
    'vehicle.car': 'car',
    'vehicle.construction': 'construction_vehicle',
    'vehicle.motorcycle': 'motorcycle',
    'human.pedestrian.adult': 'pedestrian',
    'human.pedestrian.child': 'pedestrian',
    'human.pedestrian.construction_worker': 'pedestrian',
    'human.pedestrian.police_officer': 'pedestrian',
    'movable_object.trafficcone': 'traffic_cone',
    'vehicle.trailer': 'trailer',
    'vehicle.truck': 'truck',
    'movable_object.pushable_pullable': 'DontCare',
    'movable_object.debris': 'DontCare'
}
DefaultAttribute = {
    "car": "vehicle.parked",
    "pedestrian": "pedestrian.moving",
    "trailer": "vehicle.parked",
    "truck": "vehicle.parked",
    "bus": "vehicle.parked",
    "motorcycle": "cycle.without_rider",
    "construction_vehicle": "vehicle.parked",
    "bicycle": "cycle.without_rider",
    "barrier": "",
    "traffic_cone": "",
}


# @register_dataset
class CustomNuscDataset(Dataset):
    NumPointFeatures = 5

    def __init__(self, root_path='/media/starlet/LdTho/data/sets/nuscenes/v1.0-mini', 
                 info_path=None,
                 class_names=["traffic_cone"], prep_func=None,
                 num_point_features=None):
        self.NumPointFeatures = 5
        self.class_names = class_names
        self.nusc = NuScenes(dataroot=root_path, version='v1.0-mini')
        self._prep_func = prep_func
        self.filtered_sample_tokens = []
        for sample in self.nusc.sample:
            sample_token = sample['token']
            sample_lidar_token = sample['data']['LIDAR_TOP']
            boxes = self.nusc.get_boxes(sample_lidar_token)
            for box in boxes:
                # self.box_classes.add(box.name
                if box.name not in NameMapping.keys():
                    continue
                if NameMapping[box.name] in self.class_names:
                    self.filtered_sample_tokens.append(sample_token)
                    break
        self.filtered_sample_tokens = self.filtered_sample_tokens[
                                      :round(len(self.filtered_sample_tokens) * TRAINVAL_SPLIT_PERCENTAGE)]

        self.split = np.arange(len(self.filtered_sample_tokens))

    def __len__(self):
        return self.split.shape[0]

    def __getitem__(self, index):
        input_dict = self.get_sensor_data(index)
        return input_dict

        example = self._prep_func(input_dict=input_dict)
        example["metadata"] = input_dict["metadata"]
        if "anchors_mask" in example:
            example["anchors_mask"] = example["anchors_mask"].astype(np.uint8)
        return example

    def get_sensor_data(self, query):
        res = {
            'lidar': {
                'type': 'lidar',
                'points': None,
            },
            'metadata': {
                'token': self.filtered_sample_tokens[query]
            }
        }
        points = self.getPoints(query)
        boxes_dict = self.getBoxes(query)

        res['lidar']['points'] = points

        gt_boxes = []
        gt_names = []

        for box in boxes_dict:
            xyz = box.center
            wlh = box.wlh
            theta = quaternion_yaw(box.orientation)
            gt_boxes.append([xyz[0], xyz[1], xyz[2], wlh[0], wlh[1], wlh[2], -theta - np.pi / 2])
            gt_names.append(box.name)
        gt_boxes = np.concatenate(gt_boxes).reshape(-1, 7)
        gt_names = np.array(gt_names)
        res['lidar']['annotations'] = {
            'boxes': gt_boxes,
            'names': gt_names,
        }
        return res

        ###

    def getPoints(self, index):
        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']

        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])
        calibrated_sensor = self.nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])

        global_from_car = transform_matrix(ego_pose['translation'],
                                           Quaternion(ego_pose['rotation']), inverse=False)
        car_from_sensor = transform_matrix(calibrated_sensor['translation'],
                                           Quaternion(calibrated_sensor['rotation']), inverse=False)
        try:
            lidar_pointcloud, times = LidarPointCloud.from_file_multisweep(self.nusc, sample, 'LIDAR_TOP',
                                                                           'LIDAR_TOP')
            lidar_pointcloud.transform(car_from_sensor)
        except Exception as e:
            print(f"Failed to load Lidar Pointcloud for {sample}:{e}")
        points = lidar_pointcloud.points
        points[3, :] /= 255
        points[3, :] -= 0.5

        points_cat = np.concatenate([points, times], axis=0).transpose()
        points_cat = points_cat[~np.isnan(points_cat).any(axis=1)]

        return points_cat

    def getBoxes(self, index):

        sample = self.nusc.get('sample', self.filtered_sample_tokens[index])
        sample_lidar_token = sample['data']['LIDAR_TOP']
        lidar_data = self.nusc.get('sample_data', sample_lidar_token)
        ego_pose = self.nusc.get('ego_pose', lidar_data['ego_pose_token'])

        boxes_dict = self.nusc.get_boxes(sample_lidar_token)

        keep_box_idx = []
        for i, box in enumerate(boxes_dict):
            if box.name not in NameMapping.keys():
                continue
            if NameMapping[box.name] in self.class_names:
                box.name = NameMapping[box.name]
                keep_box_idx.append(i)

        boxes_dict = [box for i, box in enumerate(boxes_dict) if i in keep_box_idx]
        self.move_boxes_to_car_space(boxes_dict, ego_pose)
        # print(boxes_dict)
        return boxes_dict

    def move_boxes_to_car_space(self, boxes, ego_pose):
        """
        Move boxes from world space to car space.
        Note: mutates input boxes.
        """
        translation = -np.array(ego_pose['translation'])
        rotation = Quaternion(ego_pose['rotation']).inverse

        for box in boxes:
            print(box)
            # Bring box to car space
            box.translate(translation)
            box.rotate(rotation)
            print("---------------")
            print(box)
            print("+++++++++++++++")


In [5]:
dset = CustomNuscDataset()

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.414 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [6]:
dset[1]

label: nan, score: nan, xyz: [410.07, 1196.78, 0.71], wlh: [0.43, 0.36, 0.79], rot axis: [0.00, 0.00, 1.00], ang(degrees): -116.06, ang(rad): -2.03, vel: nan, nan, nan, name: traffic_cone, token: e958e94168224872a04b92c4a01b8969
---------------
label: nan, score: nan, xyz: [-19.02, -6.54, 0.28], wlh: [0.43, 0.36, 0.79], rot axis: [-0.22, 0.11, 0.97], ang(degrees): -6.39, ang(rad): -0.11, vel: nan, nan, nan, name: traffic_cone, token: e958e94168224872a04b92c4a01b8969
+++++++++++++++
label: nan, score: nan, xyz: [360.80, 1124.19, 0.19], wlh: [0.42, 0.44, 0.76], rot axis: [0.00, 0.00, -1.00], ang(degrees): 23.28, ang(rad): 0.41, vel: nan, nan, nan, name: traffic_cone, token: bc1d4ea644d1481e9ec4294194d8b36e
---------------
label: nan, score: nan, xyz: [66.00, -28.19, 0.42], wlh: [0.42, 0.44, 0.76], rot axis: [0.01, -0.02, 1.00], ang(degrees): 86.60, ang(rad): 1.51, vel: nan, nan, nan, name: traffic_cone, token: bc1d4ea644d1481e9ec4294194d8b36e
+++++++++++++++
label: nan, score: nan, xyz: 

{'lidar': {'type': 'lidar',
  'points': array([[ 4.75092211e-01,  3.11872244e+00,  6.04341892e-03,
          -4.88235294e-01,  0.00000000e+00],
         [ 4.77077808e-01,  3.27951100e+00,  1.33175453e-02,
          -4.84313725e-01,  0.00000000e+00],
         [ 4.78572563e-01,  3.46075520e+00,  1.63190036e-02,
          -4.80392157e-01,  0.00000000e+00],
         ...,
         [-7.12287288e-01,  1.39881904e+01,  3.86281490e+00,
          -2.01960784e-01,  2.00189114e-01],
         [-7.00358198e-01,  1.39862118e+01,  4.19502911e+00,
          -2.25490196e-01,  2.00189114e-01],
         [-6.90844099e-01,  1.39738642e+01,  4.53035938e+00,
          -3.43137255e-01,  2.00189114e-01]]),
  'annotations': {'boxes': array([[-19.02437285,  -6.53686421,   0.28442986,   0.427     ,
             0.359     ,   0.794     ,  -1.46277525],
          [ 65.99517651, -28.19245062,   0.41549131,   0.418     ,
             0.439     ,   0.761     ,  -3.08224962],
          [  5.91182926,  -6.88653184,   0.3